In [2]:
import json
import pickle
import os

import pandas as pd
import requests
import numpy as np

import config

data_path = os.path.join("headHunter_data")

text_processing_url = config.text_processing_url

pd.options.display.max_rows = 10
pd.set_option('display.max_columns', None)

In [6]:

with open(os.path.join(data_path, "hh_ids.dat"), 'rb') as inf:
    ids = pickle.load(inf)
    
with open(os.path.join(data_path, "hh_vacancies.dat"), 'rb') as inf:
    vacancies = pickle.load(inf)
    
with open(os.path.join(data_path, "hh_vacancies_ext.dat"), 'rb') as inf:
    vacancies_ext = pickle.load(inf)
    

In [7]:
print(len(vacancies_ext))
print(len(ids))

25521
24980


In [8]:
vac_rows = []
for vac in vacancies:
    try:
        row = {"id": vac["id"], "title": vac["name"],
               "title_normalized": "",
               "title_lemmas": "",
               "title_lemmas_tags": "",
               "title_tokens": "",
               "lang_title": "",
               "requirement_norm": "",
               "requirement_lemmas": "",
               "requirement_lemmas_tags": "",
               "requirement_tokens": "",
               "responsibility_norm": "",
               "responsibility_lemmas": "",
               "responsibility_lemmas_tags": "",
               "responsibility_tokens": "",
               "requirement": vac["snippet"]["requirement"],
               "responsibility": vac["snippet"]["responsibility"],
               "url": vac["url"]}
        vac_rows.append(row) 
    except KeyError:
        print(row)
        None      
vac_df = pd.DataFrame(vac_rows)

print()

vac_rows = []
for i, vac in enumerate(vacancies_ext):
    try:
        row = {"id": vac["id"], "text": vac["description"],
               "text_normalized": "",
               "text_lemmas": "",
               "text_lemmas_tags": "",
               "text_tokens": "",
               "lang_text": "",
               "specializations": [i["name"] for i in vac["specializations"]],
               "profarea_names": [i["profarea_name"] for i in vac["specializations"]]}
        vac_rows.append(row)   
    except KeyError:
        print("Key error, index =", i, "Item =", vacancies_ext[i])
        None
vac_df_ext = pd.DataFrame(vac_rows)

vac_df.drop_duplicates(["id"], inplace=True)
vac_df_ext.drop_duplicates(["id"], inplace=True)
full_df = vac_df_ext.merge(vac_df, left_on='id', right_on='id', how='outer')

print(len(full_df))


Key error, index = 140 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 198 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 735 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1074 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1138 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1140 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1376 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1633 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1798 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1825 Item = {'description': 'Not Found', 'errors': [{'type': 'not_found'}]}
Key error, index = 1838 Item = {'description': 'Not 

In [9]:
#delete unuseful variables
del vac_rows
del vac_df_ext
del vac_df
del vacancies
del vacancies_ext
del ids

### Drop already preprocessed rows

In [10]:
exist_df = pd.read_csv(os.path.join(data_path, "hh_dataset.csv"), sep='\t')
exist_df = exist_df.loc[np.logical_or(exist_df["lang_text"]=="russian",
                                      exist_df["lang_text"]=="english")]
exist_df.reset_index(drop=True, inplace=True)
print(exist_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22795 entries, 0 to 22794
Data columns (total 26 columns):
id                            22795 non-null int64
lang_text                     22795 non-null object
lang_title                    22795 non-null object
profarea_names                22795 non-null object
requirement                   21962 non-null object
requirement_lemmas            21961 non-null object
requirement_lemmas_tags       21961 non-null object
requirement_norm              21961 non-null object
requirement_tokens            21961 non-null object
responsibility                21589 non-null object
responsibility_lemmas         21588 non-null object
responsibility_lemmas_tags    21588 non-null object
responsibility_norm           21588 non-null object
responsibility_tokens         21588 non-null object
specializations               22795 non-null object
text                          22795 non-null object
text_lemmas                   22795 non-null object
text_lem

In [11]:
exist_ids = [str(i) for i in exist_df["id"].values]
new_ids = [str(i) for i in full_df["id"].values]
print(len(exist_ids))
print(len(new_ids))

to_drop_ids = list(set(new_ids).intersection(set(exist_ids)))
print(len(to_drop_ids))

to_proc_ids = list(set(new_ids).difference(set(exist_ids)))
print(len(to_proc_ids))

22795
24980
22268
2712


In [12]:
%%time
for i in to_drop_ids:
    full_df = full_df.loc[full_df["id"] != i]  
    #full_df.drop(full_df["id"] == i], inplace=True)
    full_df.reset_index(drop=True, inplace=True)
print(len(full_df))
print(full_df.info())

2712
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712 entries, 0 to 2711
Data columns (total 26 columns):
id                            2712 non-null object
lang_text                     2710 non-null object
profarea_names                2710 non-null object
specializations               2710 non-null object
text                          2710 non-null object
text_lemmas                   2710 non-null object
text_lemmas_tags              2710 non-null object
text_normalized               2710 non-null object
text_tokens                   2710 non-null object
lang_title                    2712 non-null object
requirement                   2639 non-null object
requirement_lemmas            2712 non-null object
requirement_lemmas_tags       2712 non-null object
requirement_norm              2712 non-null object
requirement_tokens            2712 non-null object
responsibility                2568 non-null object
responsibility_lemmas         2712 non-null object
responsibility_lemmas

### Normalize text and title using text_preprocessing service

In [13]:
%%time
for index, row in full_df.iterrows():

    requirement = row["requirement"]
    responsibility = row["responsibility"]
    
    fields = ["title", "text"]
    for field in fields:
        text = row[field]
        r = requests.post(text_processing_url + config.STEM_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_normalized"] = r.text
        
        r = requests.post(text_processing_url + config.LEMM_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas"] = r.text
        
        r = requests.post(text_processing_url + config.TAG_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas_tags"] = r.text
        
        r = requests.post(text_processing_url + config.TOKEN_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_tokens"] = r.text
        
        r = requests.post(text_processing_url + config.DETECT_LANG_PATH ,
                          json=text)
        full_df.loc[index, "lang_" + field] = r.text
                
    fields = ["requirement", "responsibility"]
    for field in fields:
        text = row[field]
        r = requests.post(text_processing_url + config.STEM_TEXT_PATH,
                              json=text)
        full_df.loc[index, field + "_norm"] = r.text
        
        r = requests.post(text_processing_url + config.LEMM_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas"] = r.text
        
        r = requests.post(text_processing_url + config.TAG_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_lemmas_tags"] = r.text
        
        r = requests.post(text_processing_url + config.TOKEN_TEXT_PATH,
                          json=text)
        full_df.loc[index, field + "_tokens"] = r.text
    
    if index % 500 == 0:
        print(index)
  

0
500
1000
1500
2000
2500
Wall time: 57min 59s


In [14]:
print(len(full_df))
full_df.tail()

2712


,id,lang_text,profarea_names,specializations,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,lang_title,requirement,requirement_lemmas,requirement_lemmas_tags,requirement_norm,requirement_tokens,responsibility,responsibility_lemmas,responsibility_lemmas_tags,responsibility_norm,responsibility_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url
2707,28852822,russian,"[Информационные технологии, интернет, телеком,...","[Системный администратор, Поддержка, Helpdesk,...",<p>Привет!</p> <p><strong>Алгоритмика </strong...,привет алгоритмик международный школа программ...,привет_NOUN алгоритмик_NOUN международный_ADJ ...,привет алгоритмик международн школ программиро...,привет алгоритмика международная школа програм...,russian,Чистоплотность в работе. Отсутствия страха при...,чистоплотность работа отсутствие страх докумен...,чистоплотность_NOUN работа_NOUN отсутствие_NOU...,чистоплотн работ отсутств страх документирован...,чистоплотность работе отсутствия страха докуме...,Своевременная обработка и закрытие заявок в си...,своевременный обработка закрытие заявка систем...,своевременный_ADJ обработка_NOUN закрытие_NOUN...,своевремен обработк закрыт заявок систем helpd...,своевременная обработка закрытие заявок систем...,Специалист Helpdesk,специалист,специалист_NOUN,специалист helpdesk,специалист helpdesk,https://api.hh.ru/vacancies/28852822?host=hh.ua
2708,28941246,russian,"[Информационные технологии, интернет, телеком,...","[Начальный уровень, Мало опыта, Программирован...",<p>В связи с дальнейшим развитием бизнеса круп...,связь дальнейший развитие бизнес крупный федер...,связь_NOUN дальнейший_ADJ развитие_NOUN бизнес...,связ дальн развит бизнес крупн федеральн пригл...,связи дальнейшим развитием бизнеса крупная фед...,russian,Высшее образование (техническое). Опыт поддерж...,высший образование технический опыт поддержка ...,высший_ADJ образование_NOUN технический_ADJ оп...,высш образован техническ оп поддержк консульта...,высшее образование техническое опыт поддержки ...,Консультации пользователей по работе с програм...,консультация пользователь работа программа реш...,консультация_NOUN пользователь_NOUN работа_NOU...,консультац пользовател работ программ решен об...,консультации пользователей работе программами ...,Консультант 1С,консультант,консультант_NOUN,консультант,консультант,https://api.hh.ru/vacancies/28941246?host=care...
2709,29225980,russian,"[Бухгалтерия, управленческий учет, финансы пре...","[Бухгалтер, Учет заработной платы, Налоги, ТМЦ...",<p><strong>Обязанности:</strong></p> <ul> <li>...,обязанность ведение бухгалтерский налоговый уч...,обязанность_NOUN ведение_NOUN бухгалтерский_AD...,обязан веден бухгалтерск налогов учет работ пе...,обязанности ведение бухгалтерского налогового ...,russian,Знание бухгалтерского и налогового российского...,знание бухгалтерский налоговый российский зако...,знание_NOUN бухгалтерский_ADJ налоговый_ADJ ро...,знан бухгалтерск налогов российск законодатель...,знание бухгалтерского налогового российского з...,Ведение бухгалтерского и налогового учета. Раб...,ведение бухгалтерский налоговый учет работа пе...,ведение_NOUN бухгалтерский_ADJ налоговый_ADJ у...,веден бухгалтерск налогов учет работ первичн д...,ведение бухгалтерского налогового учета работа...,Главный бухгалтер,главный бухгалтер,главный_ADJ бухгалтер_NOUN,главн бухгалтер,главный бухгалтер,https://api.hh.ru/vacancies/29225980?host=hh.ua
2710,28875300,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 3.2 Fi...",NaN,NaN,NaN,,,,,russian,Опыт телефонных продаж от 3 лет в компаниях с ...,опыт телефонный продажа год компания труктурир...,опыт_NOUN телефонный_ADJ продажа_NOUN год_NOUN...,оп телефон продаж лет компан структурирова сис...,опыт телефонных продаж лет компаниях структури...,Формирование команды лояльных и эффективных со...,формирование команда лояльный эффективный сотр...,формирование_NOUN команда_NOUN лояльный_ADJ эф...,формирован команд лояльн эффективн сотрудник у...,формирование команды ло

In [15]:
print(full_df.loc[4, "requirement_lemmas"])
print()
print(full_df.loc[4, "responsibility_lemmas"])
print()
print(full_df.loc[4, "text_tokens"])

высший коммуникационный навык высокий ориентация ученик результат грамотный речь доброжелательность ответственность

преподавание курс скорочтений развитие интеллект школа скорочтения развитие интеллект проведение занятие согласно методика умц

школа скорочтение развития интеллекта международная сеть элитных школ которая занимается всесторонним развитием взрослых подростков детей связи расширением сети открыта вакансия преподавателя почему нам доверяют наша качественная работа также деловая репутация высоко отмечены школа скорочтение развития интеллекта признана лучшей компанией номинации лидер отрасли награждена медалью национальный знак качества удостоена почетного звания российское качество занимает рейтинге журнала forbes место обязанности преподавание курса скорочтение развитие интеллекта школе скорочтения развития интеллекта требования нужна работа нужны нам почему стоит выбрать школа скорочтения развития интеллекта это компания отмеченная наградами федерального уровня занимающая

In [16]:
print(len(exist_df))
exist_df.head()

22795


,id,lang_text,lang_title,profarea_names,requirement,requirement_lemmas,requirement_lemmas_tags,requirement_norm,requirement_tokens,responsibility,responsibility_lemmas,responsibility_lemmas_tags,responsibility_norm,responsibility_tokens,specializations,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url
0,21750144,russian,russian,"['Безопасность', 'Безопасность', 'Безопасность...","Приглашаем отслуживших в ВС РФ (группа А), име...",приглашать отслуживший воскресенье рфа группа ...,приглашать_VERB отслуживший_VERB воскресенье_N...,приглаша отслуж вс рф групп имеющ средн полн в...,приглашаем отслуживших вс рф группа имеющих ср...,"На собеседование приходить ( рубашка, брюки, т...",собеседование приходить рубашка брюки туфля со...,собеседование_NOUN приходить_VERB рубашка_NOUN...,собеседован приход рубашк брюк туфл соблюда фо...,собеседование приходить рубашка брюки туфли со...,"['Системы видеонаблюдения', 'Пожарная безопасн...",<p>работа на территории работодателя г. Москва...,работа территория работодатель москвый обязанн...,работа_NOUN территория_NOUN работодатель_NOUN ...,работ территор работодател москв обязан наш ос...,работа территории работодателя москва обязанно...,Полицейский (г. Москва),полицейский москво,полицейский_NOUN москво_NOUN,полицейск москв,полицейский москва,https://api.hh.ru/vacancies/21750144?host=hh.ua
1,28974214,russian,russian,"['Начало карьеры, студенты', 'Закупки', 'Закуп...",Свободное владение английским языком (устная и...,свободный владение английский язык устный пись...,свободный_ADJ владение_NOUN английский_ADJ язы...,свободн владен английск язык устн письмен реч ...,свободное владение английским языком устная пи...,Подготовка и регистрация международных контрак...,подготовка регистрация международный контракт ...,подготовка_NOUN регистрация_NOUN международный...,подготовк регистрац международн контракт подго...,подготовка регистрация международных контракто...,"['Закупки', 'Продукты питания', 'Начальный уро...",<p><strong>Обязанности:</strong></p> <p><stron...,обязанность помощь сопровождение импортный зак...,обязанность_NOUN помощь_NOUN сопровождение_NOU...,обязан помощ сопровожден импортн закупок рыб м...,обязанности помощь сопровождении импортных зак...,Ассистент менеджера ВЭД,ассистент менеджер вэд,ассистент_NOUN менеджер_NOUN вэд_ADV,ассистент менеджер вэд,ассистент менеджера вэд,https://api.hh.ru/vacancies/28974214?host=care...
2,28694873,russian,russian,['Продажи'],Доброжелательность и позитивный настрой. Ответ...,доброжелательность позитивный настрой ответств...,доброжелательность_NOUN позитивный_ADJ настрой...,доброжелательн позитивн настр ответствен пункт...,доброжелательность позитивный настрой ответств...,Работа с кассовым аппаратом и прикассовой зоне...,работа кассовый аппарат прикассовый зона подде...,работа_NOUN кассовый_ADJ аппарат_NOUN прикассо...,работ кассов аппарат прикассов зон поддержан р...,работа кассовым аппаратом прикассовой зоне под...,['Продавец в магазине'],"<p>В сетевые магазины Дикси, FixPrice, Воентор...",сетевой магазин дикси fixprice военторгать пят...,сетевой_ADJ магазин_NOUN дикси_NOUN fixprice_X...,сетев магазин дикс fixpric военторг пятерочк т...,сетевые магазины дикси fixprice военторг пятер...,Кассир,кассир,кассир_NOUN,кассир,кассир,https://api.hh.ru/vacancies/28694873?host=hh.ua
3,28652642,russian,russian,"['Продажи', 'Продажи', 'Продажи', 'Продажи', '...",Желание работать и развиваться в крупной компа...,желание работать развиваться крупный компания ...,желание_NOUN работать_VERB развиваться_VERB кр...,желан работа развива крупн компан внимательн о...,желание работать развиваться крупной компании ...,Своевременная выкладка товара в отделе. Отслеж...,своевременный выкладкий товар отдел отслеживан...,своевременный_ADJ выкладкий_NOUN товар_NOUN от...,своевремен выкладк товар отдел отслеживан коли...,своевременная выкладка товара отделе отслежива...,"['Розничная торговля', 'Начал

In [17]:
full_df = full_df.append(exist_df, ignore_index=True, sort=True)
print("Final size of dataset =", len(full_df))

Final size of dataset = 25507


In [18]:
full_df.drop_duplicates(["id"], inplace=True)
print("Final size of dataset =", len(full_df))

Final size of dataset = 25507


In [19]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25507 entries, 0 to 25506
Data columns (total 26 columns):
id                            25507 non-null object
lang_text                     25507 non-null object
lang_title                    25507 non-null object
profarea_names                25505 non-null object
requirement                   24601 non-null object
requirement_lemmas            24673 non-null object
requirement_lemmas_tags       24673 non-null object
requirement_norm              24673 non-null object
requirement_tokens            24673 non-null object
responsibility                24157 non-null object
responsibility_lemmas         24300 non-null object
responsibility_lemmas_tags    24300 non-null object
responsibility_norm           24300 non-null object
responsibility_tokens         24300 non-null object
specializations               25505 non-null object
text                          25505 non-null object
text_lemmas                   25507 non-null object
text_le

In [20]:
full_df.tail()

,id,lang_text,lang_title,profarea_names,requirement,requirement_lemmas,requirement_lemmas_tags,requirement_norm,requirement_tokens,responsibility,responsibility_lemmas,responsibility_lemmas_tags,responsibility_norm,responsibility_tokens,specializations,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url
25502,29103671,russian,russian,"['Управление персоналом, тренинги']",любишь работу рекрутера и готов к новым вызова...,любить работа рекрутер готовый новый вызов зна...,любить_VERB работа_NOUN рекрутер_NOUN готовый_...,люб работ рекрутер гот нов вызов знаеш английс...,любишь работу рекрутера готов новым вызовам зн...,пользоваться сайтами по поиску работы. - имееш...,пользоваться сайт поиск работа иметь хотя конт...,пользоваться_VERB сайт_NOUN поиск_NOUN работа_...,пользова сайт поиск работ имееш хот контакт пе...,пользоваться сайтами поиску работы имеешь хотя...,['Рекрутмент'],<p>Группа рекламных агентств Havas ищет рекрут...,группа рекламный агентство havas искать рекрут...,группа_NOUN рекламный_ADJ агентство_NOUN havas...,групп рекламн агентств havas ищет рекрутераесл...,группа_NOUN рекламный_ADJ агентство_NOUN havas...,Рекрутер,рекрутер,рекрутер_NOUN,рекрутер,рекрутер_NOUN,https://api.hh.ru/vacancies/29103671?host=hh.uz
25503,29103670,russian,russian,"['Бухгалтерия, управленческий учет, финансы пр...","Образование: средне-специальное, высшее. - Уве...",образование высший уверенный владение парсек w...,образование_NOUN высший_ADJ уверенный_ADJ влад...,образован высш уверен владен пк word excel гот...,образование высшее уверенное владение пк word ...,Формирование новых и контроль уже существующих...,формирование новый контроль существовать кальк...,формирование_NOUN новый_ADJ контроль_NOUN суще...,формирован нов контрол существ калькуляцион ка...,формирование новых контроль существующих кальк...,"['Бухгалтер-калькулятор', 'Начальный уровень, ...",<p>Мегамарт - это уральское отделение федераль...,мегамарт этот уральский отделение федеральный ...,мегамарт_NOUN этот_DET уральский_ADJ отделение...,мегамарт эт уральск отделен федеральн торгов с...,мегамарт_NOUN этот_DET уральский_ADJ отделение...,Бухгалтер-калькулятор (улица Луначарского),улица луначарский,улица_NOUN луначарский_ADJ,улиц луначарск,улица_NOUN луначарский_ADJ,https://api.hh.ru/vacancies/29103670?host=hh.uz
25504,27642665,russian,russian,"['Информационные технологии, интернет, телеком']",Знание продукта Microsoft Dynamics CRM (или ин...,знание продукт микрософт dynamics crm иной crm...,знание_NOUN продукт_NOUN микрософт_X dynamics_...,знан продукт microsoft dynamics crm ин crm сис...,знание продукта microsoft dynamics crm иных cr...,Участие в проекте внедрения системы Microsoft ...,участие проект внедрение система микрософт dyn...,участие_NOUN проект_NOUN внедрение_NOUN систем...,участ проект внедрен систем microsoft dynamics...,участие проекте внедрения системы microsoft dy...,['Аналитик'],<p>ПАО «Промсвязьбанк» – один из крупнейших ро...,пао промсвязьбанкать крупный российский банк в...,пао_NOUN промсвязьбанкать_NOUN крупный_ADJ рос...,па промсвязьбанк крупн российск банк входя бан...,пао_NOUN промсвязьбанкать_NOUN крупный_ADJ рос...,Аналитик Dynamics CRM,аналитик dynamics crm,аналитик_NOUN dynamics_X crm_X,аналитик dynamics crm,аналитик_NOUN dynamics_X crm_X,https://api.hh.ru/vacancies/27642665?host=hh.uz
25505,29103668,russian,russian,"['Маркетинг, реклама, PR']",Вам необходимо добиваться поставленного резуль...,необходимый добиваться поставить результат име...,необходимый_ADJ добиваться_VERB поставить_VERB...,необходим добива поставлен результат имет высш...,необходимо добиваться поставленного результата...,Основная обязанность запускать и вести рекламн...,основной обязанность запускать вести рекламный...,основной_ADJ обязанность_NOUN запускать_VERB в...,основн обязан запуска вест рекламн кампан янде...,основная обязанность запускать вести рекламные...,['Интернет-маркетинг'],<p>Ищем 

In [21]:
full_df.to_csv(os.path.join("hh_dataset.csv"),
               sep='\t', header=True, index=None)

In [ ]:
vacancies[0]

In [ ]:
vacancies_ext[0]